#### Mapa de kills no League of Legends com Datashader
> Importar os pacotes necessários para que seja plotado o mapa com os abates dos jogadores

In [3]:
import pandas as pd
import datashader as ds
from datashader import transfer_functions as tf
import numpy as np

> DataFrame com os registros de abates do arquivo csv e a função .head()

In [5]:
df = pd.read_csv('kills.csv.zip', compression='zip')
df.head()

,Address,Team,Time,Victim,Killer,Assist_1,Assist_2,Assist_3,Assist_4,x_pos,y_pos
0,http://matchhistory.na.leagueoflegends.com/en/...,bKills,10.820,C9 Hai,TSM Bjergsen,NaN,NaN,NaN,NaN,9229,8469
1,http://matchhistory.na.leagueoflegends.com/en/...,bKills,16.286,C9 LemonNation,TSM WildTurtle,TSM Santorin,TSM Bjergsen,TSM Lustboy,NaN,13908,5960
2,http://matchhistory.na.leagueoflegends.com/en/...,bKills,18.733,C9 Hai,TSM Bjergsen,TSM Santorin,TSM WildTurtle,TSM Lustboy,NaN,8281,7889
3,http://matchhistory.na.leagueoflegends.com/en/...,bKills,18.880,C9 Meteos,TSM Dyrus,TSM Santorin,TSM Bjergsen,TSM WildTurtle,TSM Lustboy,8522,8895
4,http://matchhistory.na.leagueoflegends.com/en/...,bKills,27.005,C9 Balls,TSM Bjergsen,TSM Dyrus,TSM Santorin,TSM WildTurtle,TSM Lustboy,8902,7643


> Trabalharemos com as colunas de tempo (time) e os eixos x (x_pos) e y (y_pos).

In [6]:
df = df.drop(['Address', 'Team', 'Victim', 'Killer', 'Assist_1', 'Assist_2', 'Assist_3', 'Assist_4'], axis=1)
df = df.rename(columns={'Time': 'time'})
df = df.loc[df.x_pos != 'TooEarly']
df = df.loc[df.x_pos.notnull()]
df.x_pos = df.x_pos.apply(lambda x: int(x))
df.y_pos = df.y_pos.apply(lambda y: int(y))

> Os abates estão sendo concentrados em 3 momentos, até os 16 minutos onde podemos estimar que seja o early game, até os 24 minutos onde podem estar os abates que ocorreram em middle game e a cima dos 32 minutos os abates que ocorreram no late game.

In [7]:
df.describe()

,time,x_pos,y_pos
count,190823.000000,190823.000000,190823.000000
mean,24.545016,7553.509933,7591.560834
std,11.152311,3692.012505,3661.199082
min,0.845000,65.000000,185.000000
25%,16.018500,4692.000000,4875.000000
50%,24.637000,7483.000000,7729.000000
75%,32.174500,10506.000000,10349.000000
max,94.536000,14650.000000,14644.000000


> A função create_phase_category retornará o DataFrame atualizado com uma nova coluna: phase. Essa coluna é do tipo category e possuí três tipos diferentes: early, mid e late.

In [9]:
def create_phase_category(df):
    conditions = [
        (df.time<17),
        (17<df.time) & (df.time<32),
        (32<df.time)]

    choices = ['early', 'mid', 'late']
    df['phase'] = np.select(conditions, choices, default='very_early')
    df['phase'] = df['phase'].astype('category')
    
    return(df)
  
df = create_phase_category(df)

> A função visualize_with_datashader tem um dicionário que define as cores dos pontos para cada estação de tempo do jogo, onde: early é lightyellow, mid é tomato e late é firebrick.

In [12]:
def visualise_with_datashader(df):
    color_key = {'very_early': 'black', 'early': 'lightyellow', 'mid': 'tomato', 'late': 'firebrick'}
    
    cvs = ds.Canvas()
    agg = cvs.points(df, 'x_pos', 'y_pos', ds.count_cat('phase'))
    
    img = tf.shade(agg, color_key=color_key, how='eq_hist')
    img = tf.set_background(img, 'black')
    
    return(img)
  
img = visualise_with_datashader(df)
img.show()

AttributeError: 'Image' object has no attribute 'show'